# Importing required libraries

In [1]:
import sys
import pprint
import pandas as pd
import numpy as np
from scipy.special import softmax

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import ParameterGrid
import sklearn
import os

import shutil

from os import listdir
from os.path import isfile, join

import time

In [2]:
# Setting path for importing required functions for data processing

sys.path.append("/home/jupyter/sonam/adhd_nlp/final_notebook_folder/data_processing")
sys.path.append("/home/jupyter/sonam/adhd_nlp/final_notebook_folder/data_processing")

In [3]:
# Functions required for data processing

import final_process_text
import final_transform_textfiles

In [4]:
pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 300)
pd.set_option('display.max_columns', 100)

In [5]:
# using simpletransformer ai library

from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [6]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Import Data

In [7]:
deploy_df = pd.read_csv('/home/jupyter/data/cohort_2to6/notes-other-adhd-wc-visits-2021-10-19.csv')  

In [8]:
deploy_df.shape

(1023, 4)

In [10]:
# checking for null values

deploy_df['note_des'].isna().sum()

3

In [11]:
deploy_df = deploy_df.dropna()

In [12]:
#deploy_df.shape
deploy_df = deploy_df.reset_index(drop=True)
print("The number of samples in deployment set are ",deploy_df.shape[0])
#deploy_df.tail(2)

The number of samples in deployment set are  1020


# Data Processing 

In [14]:
deploy_df['extractText'] = deploy_df['note_des'].apply(lambda x: final_process_text.sectionize(x)[1]) 

In [15]:
deploy_df['extractText'] = deploy_df['extractText'].apply(lambda x: final_process_text.clean_text(x)) 

In [19]:
deploy_df= deploy_df.loc[:, ['ANON_ID','PAT_ENC_ID_JITTERED','extractText']]\
       .rename(columns = {'extractText':'text'})

In [20]:
#deploy_df.tail(2)
print("The final deploy_df looks like below.")
deploy_df.dtypes

The final deploy_df looks like below.


ANON_ID                 int64
PAT_ENC_ID_JITTERED     int64
text                   object
dtype: object

# Loading saved model weights and getting results for deployment set

In [21]:
# loading saved weights from the training  for defining the model
model = ClassificationModel("bert", "./final_biobert_output_dir_new_cohort",use_cuda = True, num_labels = 2)

In [22]:
# Predicting labels and probablities for 1020 samples and adding in deploy_df as columns
predictions, probabilities = model.predict(deploy_df['text'].tolist())

deploy_df['predictions'] = predictions
deploy_df['probabilities'] = [x[1] for x in np.array([softmax(element) for element in probabilities])]

In [24]:
#deploy_df.tail(2)
print("The deploy_df dataframe looks like below now.")
deploy_df.dtypes

The deploy_df dataframe looks like below now.


ANON_ID                  int64
PAT_ENC_ID_JITTERED      int64
text                    object
predictions              int64
probabilities          float64
dtype: object

In [25]:
deploy_pred_prob = deploy_df['probabilities']

# threshold value obtained from Final_423_Deployment_notebook_new_cohort.ipnyb
threshold_final = 0.001842

In [26]:
# function to obtain the prediction using threshold value for probabilities
def pred_label(threshold_final, pred_prob):
    
    pred_label = (pred_prob >= threshold_final)

    pred_label = pred_label.values.astype(int)
    
    return pred_label

In [27]:
#Passing threshold and probabilities to get prediction according to threshold
pred_prob_arr = pred_label(threshold_final, deploy_pred_prob)

In [29]:
# updating predictions column with predictions obtained after using threshold values for probabilities
deploy_df['predictions']= pred_prob_arr

#Cross Checking Results for predictions with threshold and without threshold.

In [30]:
type(predictions)
print("Without Threshold:")
print("predicted 0:", predictions.count(0))
print("predicted 1:", predictions.count(1))

Without Threshold:
predicted 0: 975
predicted 1: 45


In [31]:
pred_prob_label_list =deploy_df['predictions'].tolist()

print("With Threshold of 0.001842:")
print("predicted 0:", pred_prob_label_list.count(0))
print("predicted 1:", pred_prob_label_list.count(1))

With Threshold of 0.001842:
predicted 0: 970
predicted 1: 50


# Saving the file having "ANNON_ID", "ENC_ID",text" and "label"

In [32]:
header = ["ANON_ID", "PAT_ENC_ID_JITTERED","predictions", "text"]
deploy_df.to_csv('/home/jupyter/sonam/final_result_files/final_biobert_output_1020notes.csv', columns = header, index=False)
print("The output csv file is saved with predictions for the deployment notes.")

The output csv file is saved with predictions for the deployment notes.


In [33]:
# reading output_csv file to see if its saved correctly

deploy_results =pd.read_csv('/home/jupyter/sonam/final_result_files/final_biobert_output_1020notes.csv') 
deploy_results.shape

(1020, 4)